---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [220]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.ensemble import GradientBoostingClassifier

In [358]:
train_df = pd.read_csv("train.csv", encoding = "ISO-8859-1", low_memory=False).set_index("ticket_id")
train_df['compliance'].fillna(value = 0, inplace=True)
address_df =  pd.read_csv('addresses.csv')
coordinate_df = pd.read_csv('latlons.csv')
coordinate_df.loc[4126,'lat'] = 42.376720
coordinate_df.loc[4126,'lon'] = -83.143211
coordinate_df.loc[10466,'lat'] = 42.446541
coordinate_df.loc[10466,'lon'] = -83.023300
coordinate_df.loc[17293,'lat'] = 42.359909
coordinate_df.loc[17293,'lon'] = -83.095840
coordinate_df.loc[34006,'lat'] = 42.358910
coordinate_df.loc[34006,'lon'] = -83.151329
coordinate_df.loc[55750,'lat'] = 42.359718
coordinate_df.loc[55750,'lon'] = -83.080193
coordinate_df.loc[74721,'lat'] = 42.383251
coordinate_df.loc[74721,'lon'] = -83.058189
coordinate_df.loc[100359,'lat'] = 42.328590
coordinate_df.loc[100359,'lon'] = -83.051460
ac_df = pd.merge(address_df, coordinate_df, left_on= 'address', right_on='address', how='inner').set_index("ticket_id")
train_df = pd.merge(train_df, ac_df, left_index=True, right_index=True, how='inner')
train_df.drop(['inspector_name','violator_name', 'violation_street_number', 'violation_street_name', 'violation_zip_code',
               'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state','zip_code', 'non_us_str_code',
               'country','violation_code','violation_description', 'admin_fee','state_fee','clean_up_cost','payment_status',
               'collection_status', 'grafitti_status','compliance_detail','address','balance_due','payment_date',
               'payment_amount'], axis=1,inplace=True)

train_df.ticket_issued_date = [i.split(' ')[0] for i in train_df.ticket_issued_date]
train_df.hearing_date = [i.split(' ')[0] if i is not np.NaN else '9999-12-31' for i in train_df.hearing_date]
train_df['day_diff'] = list(map(lambda x, y: (datetime.strptime(x, '%Y-%m-%d')-datetime.strptime(y, '%Y-%m-%d')).days,train_df.hearing_date, train_df.ticket_issued_date))
train_df.drop(['ticket_issued_date', 'hearing_date'], axis=1, inplace=True)
train_df.dropna(inplace=True)
scaler1 = MinMaxScaler().fit(np.array(train_df.lat).reshape(-1, 1))
scaler2 = MinMaxScaler().fit(np.array(train_df.lon).reshape(-1, 1))
train_df.lat = scaler1.fit_transform(np.array(train_df.lat).reshape(-1, 1))
train_df.lon = scaler2.fit_transform(np.array(train_df.lon).reshape(-1, 1))

test_df = pd.read_csv("test.csv").set_index("ticket_id")
# train_df['compliance'].fillna(value = 0, inplace=True)
test_df = pd.merge(test_df, ac_df, left_index=True, right_index=True, how='inner')
test_df.drop(['inspector_name','violator_name', 'violation_street_number', 'violation_street_name', 'violation_zip_code',
               'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state','zip_code','non_us_str_code',
               'country','violation_code','violation_description', 'admin_fee','state_fee','clean_up_cost','address',
               'grafitti_status'], axis=1,inplace=True)
test_df.ticket_issued_date = [i.split(' ')[0] for i in test_df.ticket_issued_date]
test_df.hearing_date = [i.split(' ')[0] if i is not np.NaN else '9999-12-31' for i in test_df.hearing_date]
test_df['day_diff'] = list(map(lambda x, y: (datetime.strptime(x, '%Y-%m-%d')-datetime.strptime(y, '%Y-%m-%d')).days,test_df.hearing_date, test_df.ticket_issued_date))
test_df.drop(['ticket_issued_date', 'hearing_date'], axis=1, inplace=True)

test_df.lat = scaler1.fit_transform(np.array(test_df.lat).reshape(-1, 1))
test_df.lon = scaler2.fit_transform(np.array(test_df.lon).reshape(-1, 1))



agency_name_dct = {'Buildings, Safety Engineering & Env Department':1,
                  'Department of Public Works':2,
                  'Health Department':3,
                  'Detroit Police Department':4,
                  'Neighborhood City Halls':5}
disposition_dct={'Responsible by Default':1,
                 'Responsible - Compl/Adj by Default':1,
                 'Responsible by Admission':2,
                 'Responsible by Determination':3,
                 'Responsible - Compl/Adj by Determi':3,
                 'Responsible (Fine Waived) by Deter':4,
                 "Responsible (Fine Waived) by Admis":4,
                 'Responsible by Dismissal':5,
                 'Not responsible by Dismissal':0,
                 'Not responsible by City Dismissal':0,
                 'Not responsible by Determination':0,
                 'PENDING JUDGMENT':0,
                 'SET-ASIDE (PENDING JUDGMENT)':0
                 
                 
                 
                 
                }
train_df.replace({'agency_name':agency_name_dct, 'disposition': disposition_dct}, inplace=True)
test_df.replace({'agency_name':agency_name_dct, 'disposition': disposition_dct}, inplace=True)

In [362]:
train_df.disposition.value_counts()

1    138340
0     90425
2     13701
3      7644
4       195
Name: disposition, dtype: int64

In [363]:
y_train = train_df.compliance
x_train = train_df.drop('compliance', axis=1)
x_test = test_df
clf = GradientBoostingClassifier(random_state=0, learning_rate = 0.1,max_depth = 4)
clf.fit(x_train, y_train)
y_p = clf.predict_proba(x_test)[:,1]
test_df['compliance'] = y_p
test_df.compliance

ticket_id
284932    0.069846
285362    0.031968
285361    0.140304
285338    0.056916
285346    0.140304
285345    0.056916
285347    0.163906
285342    0.926058
285530    0.026241
284989    0.034254
285344    0.163906
285343    0.031968
285340    0.031968
285341    0.163906
285349    0.140304
285348    0.056916
284991    0.034254
285532    0.018643
285406    0.021722
285001    0.030774
285006    0.045766
285405    0.031968
285337    0.021722
285496    0.163906
285497    0.056916
285378    0.031968
285589    0.021722
285585    0.056916
285501    0.146156
285581    0.031968
285583    0.056916
285372    0.031968
285470    0.163906
285475    0.163906
285370    0.031968
285503    0.163906
285502    0.031968
285411    0.021722
285498    0.280231
285414    0.021722
285484    0.056916
285499    0.163906
285419    0.056916
288210    0.410672
285351    0.926599
288200    0.185034
285353    0.140304
285364    0.163906
288206    0.410672
288135    0.044558
285366    0.163906
288174    0.047869
28

In [342]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
y_train = train_df.compliance
x_train = train_df.drop('compliance', axis=1)
clf = GradientBoostingClassifier(random_state=0)
grid_values = {'learning_rate': [0.1],'max_depth':[4,5,6,7,8]}
grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(x_train, y_train)
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

C:\Users\wangh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Grid best parameter (max. AUC):  {'learning_rate': 0.1, 'max_depth': 4}
Grid best score (AUC):  0.8696274751037028
